# API_행정동단위(내국인)

In [1]:
import pandas as pd
import numpy as np
import pandasql as pdsql
import re

import datetime, calendar, timedelta, time
from collections import OrderedDict

import json
from bs4 import BeautifulSoup
import requests
import urllib

In [2]:
# Date list
year = int(input('Insert year ex)2020'))
month = input('Insert month ex)04, 12')
month_int = int(month)

lastday = calendar.monthrange(year, month_int)[1]
dt_list = list()

for day in range(1, lastday+1):
    dt = datetime.date(year,month_int,day)
    dt = dt.strftime("%Y%m%d")
    dt_list.append(dt)

len(dt_list)

Insert year ex)2020 2020
Insert month ex)04, 12 11


30

In [3]:
# Setting Variable
key = '***'
start_idx = 1
end_idx = 100
d_date = dt_list[1]
url = 'http://openapi.seoul.go.kr:8088/{}/json/SPOP_LOCAL_RESD_DONG/1/100/{}'.format(key, d_date)
# Sample
url

'http://openapi.seoul.go.kr:8088/795879516d736b643639784753666e/json/SPOP_LOCAL_RESD_DONG/1/100/20201102'

In [4]:

# Response to dict
response = urllib.request.urlopen(url)
json_str = response.read()
json_object = json.loads(json_str)

# Total count
list_total_count = json_object['SPOP_LOCAL_RESD_DONG']['list_total_count']
list_total_count

10176

In [5]:
# url_list
url_list = list()

for day in dt_list:
    d_date = str(day)
    start_idx = 1
    end_idx = 100
    
    for i in range(0,int(list_total_count/end_idx)+1):
        if end_idx <= list_total_count:
            url = 'http://openapi.seoul.go.kr:8088/{}/json/SPOP_LOCAL_RESD_DONG/{}/{}/{}'.format(key, start_idx, end_idx, d_date)
            url_list.append(url)
        else:
            end_idx = list_total_count
            url = 'http://openapi.seoul.go.kr:8088/{}/json/SPOP_LOCAL_RESD_DONG/{}/{}/{}'.format(key, start_idx, end_idx, d_date)
            url_list.append(url)
            break            
        start_idx += 100
        end_idx += 100             
            
len(url_list)

3060

In [6]:
url_list[-5:]

['http://openapi.seoul.go.kr:8088/795879516d736b643639784753666e/json/SPOP_LOCAL_RESD_DONG/9701/9800/20201130',
 'http://openapi.seoul.go.kr:8088/795879516d736b643639784753666e/json/SPOP_LOCAL_RESD_DONG/9801/9900/20201130',
 'http://openapi.seoul.go.kr:8088/795879516d736b643639784753666e/json/SPOP_LOCAL_RESD_DONG/9901/10000/20201130',
 'http://openapi.seoul.go.kr:8088/795879516d736b643639784753666e/json/SPOP_LOCAL_RESD_DONG/10001/10100/20201130',
 'http://openapi.seoul.go.kr:8088/795879516d736b643639784753666e/json/SPOP_LOCAL_RESD_DONG/10101/10176/20201130']

## Extract Data

In [7]:
df_raw_list = list()

for url_i in url_list:
    response = urllib.request.urlopen(url_i)
    json_str = response.read()
    json_object = json.loads(json_str)

    #Extract dict
    dict_raw = json_object['SPOP_LOCAL_RESD_DONG']['row']
    #dict to df
    df_raw = pd.DataFrame(dict_raw)
    df_raw_list.append(df_raw)
    #print(url_i)

len(df_raw_list)

3060

In [8]:
df_raw = pd.concat(df_raw_list, axis=0).reset_index(drop=True)
print(df_raw.shape)
df_raw.head()

(305280, 32)


,STDR_DE_ID,TMZON_PD_SE,ADSTRD_CODE_SE,TOT_LVPOP_CO,MALE_F0T9_LVPOP_CO,MALE_F10T14_LVPOP_CO,MALE_F15T19_LVPOP_CO,MALE_F20T24_LVPOP_CO,MALE_F25T29_LVPOP_CO,MALE_F30T34_LVPOP_CO,...,FEMALE_F25T29_LVPOP_CO,FEMALE_F30T34_LVPOP_CO,FEMALE_F35T39_LVPOP_CO,FEMALE_F40T44_LVPOP_CO,FEMALE_F45T49_LVPOP_CO,FEMALE_F50T54_LVPOP_CO,FEMALE_F55T59_LVPOP_CO,FEMALE_F60T64_LVPOP_CO,FEMALE_F65T69_LVPOP_CO,FEMALE_F70T74_LVPOP_CO
0,20201101,00,11110515,14023.3041,515.49,328.4716,497.2909,486.2887,500.7132,419.4391,...,439.0448,473.2326,577.1653,688.8372,744.7797,572.8557,566.621,448.2972,295.1666,1143.8546
1,20201101,00,11110530,15101.1307,400.3455,177.9312,259.5215,375.8703,531.1295,608.37,...,597.4471,716.195,775.7424,688.4835,679.8259,575.875,597.4889,462.9553,354.2679,1221.5534
2,20201101,00,11110540,4382.2972,66.4982,50.9533,123.0147,386.5837,174.7392,163.6085,...,109.0348,168.5665,186.0513,177.0344,194.4253,144.6869,192.2784,179.6866,142.5765,301.6359
3,20201101,00,11110550,15273.0998,371.8496,288.3268,434.9621,490.8724,395.5726,415.1845,...,430.6946,390.1769,598.4306,608.2721,845.3047,730.1258,676.3005,648.6966,415.2055,1478.8808
4,20201101,00,11110560,19420.8878,579.5608,419.2151,651.2301,569.3071,543.2471,389.6004,...,549.2769,500.3394,754.4474,743.4825,930.4405,814.4155,1018.3823,833.7663,611.4621,1781.4183


In [9]:
query =  """select STDR_DE_ID, count(*)
from df_raw
group by STDR_DE_ID
order by STDR_DE_ID"""

pdsql.sqldf(query)

,STDR_DE_ID,count(*)
0,20201101,10176
1,20201102,10176
2,20201103,10176
3,20201104,10176
4,20201105,10176
5,20201106,10176
6,20201107,10176
7,20201108,10176
8,20201109,10176
9,20201110,10176


## Unload

In [10]:
# Unload by file type
path_out = 'C:/Users/sky/DataAnalytics/result_table/'
file_name = '서울시열린데이터광장_생활인구_행정동단위(내국인)_{}{}'.format(year, month)
file_type = ['.txt', '.xlsx']

for type in file_type:
    if type == '.txt':
        path = path_out+file_name+type
        df_raw.to_csv(path, sep='\t', index=False, encoding='UTF8')
    elif type == '.xlsx':
        path = path_out+file_name+type
        df_raw.to_excel(path, index=False)
    else:
        print('Wrong file type')

# path_out + file_name + file_type[0]
# df_raw.to_csv('C:/Users/sky/DataAnalytics/result_table/서울시열린데이터광장_생활인구_자치구단위(내국인)_202004.txt', sep='\t', index=False, encoding='UTF8')
# df_raw.to_excel('C:/Users/sky/DataAnalytics/result_table/서울시열린데이터광장_생활인구_자치구단위(내국인)_202004.xlsx', index=False)

In [11]:
path

'C:/Users/sky/DataAnalytics/result_table/서울시열린데이터광장_생활인구_행정동단위(내국인)_202011.xlsx'